In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2

import pandas as pd

In [2]:
p = Project.objects.get(pk=193)
docs = Doc.objects.filter(query=7638, content__iregex="\w").order_by('id')
q = Query.objects.get(pk=7638)

In [3]:

docs = Doc.objects.filter(query=q,content__iregex="\w")
doids = DocOwnership.objects.filter(query=q,relevant__gt=0).values_list('doc__id',flat=True)
docs = docs.exclude(pk__in=doids)


In [5]:
from mordecai import Geoparser
geo = Geoparser()

geo.geoparse("I travelled from Oxford Circus to London Bridge, via Bank")

[{'country_conf': 0.611725,
  'country_predicted': 'GBR',
  'geo': {'admin1': 'England',
   'country_code3': 'GBR',
   'feature_class': 'S',
   'feature_code': 'BDG',
   'geonameid': '6619889',
   'lat': '51.50821',
   'lon': '-0.08763',
   'place_name': 'London Bridge'},
  'spans': [{'end': 47, 'start': 34}],
  'word': 'London Bridge'}]

In [7]:
tag, created = Tag.objects.get_or_create(
    query=q,
    title="Probably Africa",
    text="Manually assigned"
)

for d in docs:
    continent = "None"
    gp = geo.geoparse(d.content + " " d.title)
    for p in gp:
        if "geo" in p:
            try:
                a2 = country_alpha3_to_country_alpha2(p["geo"]["country_code3"])
                continent = country_alpha2_to_continent_code(a2)
                if continent=="AF":
                    d.tag.add(tag)
            except:
                pass
    
    #places = [e for e in doc.ents if e.label_ in ["GPE","LOC"]]
    #for p in places:
        #print(p, p.label_)

In [8]:
tag.update_tag()